# POS tagging and NER

Upload file

In [14]:
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Assuming the uploaded file is a CSV
  try:
    df_lion = pd.read_csv(io.BytesIO(uploaded[fn]))
    print("Successfully loaded CSV into df_lion")
  except pd.errors.ParserError:
      print(f"Error: Could not parse {fn} as a CSV. Please upload a valid CSV file.")
      df_lion = pd.DataFrame() # Create an empty DataFrame if parsing fails
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    df_lion = pd.DataFrame()

Saving cleaned_article.csv to cleaned_article (1).csv
User uploaded file "cleaned_article (1).csv" with length 2729372 bytes
Successfully loaded CSV into df_lion


In [15]:
print(df_lion.columns)

Index(['Kategori', 'Link', 'article_title', 'article_text', 'domain',
       'cleaned_article_text', 'polarity', 'subjectivity', 'tokens',
       'tokens_no_stop', 'tokens_final', 'tokens_stemmed', 'text_final',
       'sentiment_label'],
      dtype='object')


In [16]:
df_lion

,Kategori,Link,article_title,article_text,domain,cleaned_article_text,polarity,subjectivity,tokens,tokens_no_stop,tokens_final,tokens_stemmed,text_final,sentiment_label
0,Finance,https://industri.kontan.co.id/news/pemerintah-...,"Pemerintah Resmi Turunkan Harga Tiket Pesawat,...",Reporter: Leni Wandira | Editor: Wahyu T.Rahma...,kontan,kontancoid jakarta lion group mendukung penuh...,0.089297,0.302210,"['kontancoid', 'jakarta', 'lion', 'group', 'me...","['kontancoid', 'jakarta', 'lion', 'group', 'me...","['kontancoid', 'jakarta', 'lion', 'group', 'me...","['kontancoid', 'jakarta', 'lion', 'group', 'du...",kontancoid jakarta lion group dukung penuh bij...,Positive
1,Incident,https://news.okezone.com/read/2023/10/29/337/2...,Peristiwa 29 Oktober : Pesawat Lion Air Jatuh ...,SEJUMLAH peristiwa terjadi pada 29 Oktober. Sa...,okezone,sejumlah peristiwa terjadi pada oktober salah...,0.134921,0.250794,"['sejumlah', 'peristiwa', 'terjadi', 'pada', '...","['peristiwa', 'oktober', 'salah', 'satunya', '...","['peristiwa', 'oktober', 'salah', 'satunya', '...","['peristiwa', 'oktober', 'salah', 'satu', 'jat...",peristiwa oktober salah satu jatuh pesawat lio...,Positive
2,Incident,https://www.cnnindonesia.com/ekonomi/202306081...,Deret Masalah Penerbangan Lion Air Group Sepan...,Sekretaris Umum PP Muhammadiyah Abdul Mu'ti me...,cnnindonesia,sekretaris umum pp muhammadiyah abdul muti men...,0.033559,0.340945,"['sekretaris', 'umum', 'pp', 'muhammadiyah', '...","['sekretaris', 'pp', 'muhammadiyah', 'abdul', ...","['sekretaris', 'pp', 'muhammadiyah', 'abdul', ...","['sekretaris', 'pp', 'muhammadiyah', 'abdul', ...",sekretaris pp muhammadiyah abdul muti kritik p...,Netral
3,Incident,https://www.antaranews.com/video/3366141/ini-d...,Ini dugaan penyebab kecelakaan pesawat Lion Ai...,Copyright © ANTARA 2023\nDilarang keras mengam...,antaranews,dilarang keras mengambil konten melakukan craw...,0.000000,0.000000,"['dilarang', 'keras', 'mengambil', 'konten', '...","['dilarang', 'keras', 'mengambil', 'konten', '...","['dilarang', 'keras', 'mengambil', 'konten', '...","['larang', 'keras', 'ambil', 'konten', 'crawli...",larang keras ambil konten crawling indeks otom...,Netral
4,Operational,https://haji.kemenag.go.id/v5/detail/tingkatka...,"Tingkatkan Kualitas Layanan, Kemenag - Lion Ai...",21 Feb 2025 oleh Husni Anggoro | dilihat 42259...,kemenag,feb oleh husni anggoro dilihat kali\njakar...,0.144516,0.421348,"['feb', 'oleh', 'husni', 'anggoro', 'dilihat',...","['feb', 'husni', 'anggoro', 'kali', 'jakarta',...","['feb', 'husni', 'anggoro', 'kali', 'jakarta',...","['feb', 'husni', 'anggoro', 'kali', 'jakarta',...",feb husni anggoro kali jakarta phu direktorat ...,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,NaN,https://news.okezone.com/read/2018/11/03/337/1...,Penjelasan Mantan Pilot Terkait Hancurnya Pesa...,\nJAKARTA - Mantan pilot yang juga Purnawirawa...,okezone,jakarta mantan pilot yang juga purnawirawan k...,-0.056875,0.360000,"['jakarta', 'mantan', 'pilot', 'yang', 'juga',...","['jakarta', 'mantan', 'pilot', 'purnawirawan',...","['jakarta', 'mantan', 'pilot', 'purnawirawan',...","['jakarta', 'mantan', 'pilot', 'purnawirawan',...",jakarta mantan pilot purnawirawan polisi kapte...,Negative
149,NaN,https://www.antaranews.com/berita/481275/lion-...,Lion Air klaim finansial maskapai masih stabil...,\nDua hari lalu terjadi kekacauan masif di pul...,antaranews,dua hari lalu terjadi kekacauan masif di puluh...,0.212675,0.561975,"['dua', 'hari', 'lalu', 'terjadi', 'kekacauan'...","['kekacauan', 'masif', 'puluhan', 'bandara', '...","['kekacauan', 'masif', 'puluhan', 'bandara', '...","['kacau', 'masif', 'puluh', 'bandara', 'tanah'...",kacau masif puluh bandara tanah air akibat lam...,Positive
150,Finance,https://www.kompasiana.com/yupiter/5cfdd6ff0d8...,"Lion Air Kesulitan Keuangan, Tanda-tanda Perus...","\n |Belajar, Mengajar dan Menul...",kompasiana,belajar mengajar dan menulis mengantar pikira...,0.123657,0.462487,"['belajar', '

Install library and dependency

In [17]:
!pip install nlp-id transformers torch tqdm
from nlp_id.postag import PosTag
from transformers import pipeline
from tqdm import tqdm
from collections import Counter, defaultdict
tqdm.pandas()

In [18]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [19]:
# Initialize models
pos_tagger = PosTag()
ner_model = pipeline("ner", model="cahya/bert-base-indonesian-NER", grouped_entities=True)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.6.1. This might lead to b

Pos Tagging with nlp-id (normalized stemmed)

In [20]:
# Make sure the text column is string
df_lion["text_final"] = df_lion["text_final"].astype(str)

# Apply POS tagging
df_lion["pos_tags"] = df_lion["text_final"].progress_apply(lambda x: pos_tagger.get_pos_tag(x))


100%|██████████| 153/153 [00:04<00:00, 31.49it/s]


In [21]:
df_lion[["text_final", "pos_tags"]].head(10)

,text_final,pos_tags
0,kontancoid jakarta lion group dukung penuh bij...,"[(kontancoid, NN), (jakarta, NN), (lion, NN), ..."
1,peristiwa oktober salah satu jatuh pesawat lio...,"[(peristiwa, NN), (oktober, FW), (salah, JJ), ..."
2,sekretaris pp muhammadiyah abdul muti kritik p...,"[(sekretaris, NN), (pp, NN), (muhammadiyah, JJ..."
3,larang keras ambil konten crawling indeks otom...,"[(larang, NN), (keras, JJ), (ambil, VB), (kont..."
4,feb husni anggoro kali jakarta phu direktorat ...,"[(feb, NN), (husni, VB), (anggoro, NN), (kali,..."
5,historical flight lion air bawa jalan lion ai...,"[(historical, NN), (flight, FW), (lion, NN), (..."
6,adil negeri pn padang denda maskapai lion air ...,"[(adil, JJ), (negeri, NN), (pn, NN), (padang, ..."
7,gelang emas cincin berat gram duga hilang curi...,"[(gelang, NN), (emas, NN), (cincin, NN), (bera..."
8,tumpang histeris hilang emas bandara karyawan ...,"[(tumpang, VB), (histeris, JJ), (hilang, VB), ..."
9,januari wib dengar artikel tempoco jakarta lio...,"[(januari, VB), (wib, NN), (dengar, VB), (arti..."


In [22]:
# --- Analisis Frekuensi POS (Part-of-Speech) ---

# Buat dictionary untuk menyimpan Counter untuk setiap tag POS
pos_counts = defaultdict(Counter)

# Iterasi setiap baris di DataFrame
# df_lion['pos_tags'] berisi list of tuples, misal: [('lion', 'NN'), ('air', 'NN')]
for list_of_tags in df_lion['pos_tags']:
    for word, tag in list_of_tags:
        pos_counts[tag][word] += 1

print("--- Frekuensi Top 5 Kata per Jenis POS ---")

# Tampilkan 5 kata paling umum untuk setiap tag POS yang relevan
# Anda bisa ganti daftar ini sesuai dengan tag yang paling menarik
# (NN=Noun, VB=Verb, JJ=Adjective, RB=Adverb, NNP=Proper Noun)
relevant_pos_tags = ['NN', 'VB', 'JJ', 'RB', 'NNP']

for tag in relevant_pos_tags:
    if tag in pos_counts:
        print(f"\nTop 5 untuk tag '{tag}':")
        print(pos_counts[tag].most_common(5))

--- Frekuensi Top 5 Kata per Jenis POS ---

Top 5 untuk tag 'NN':
[('pesawat', 1042), ('air', 962), ('lion', 953), ('bandara', 440), ('udara', 216)]

Top 5 untuk tag 'VB':
[('terbang', 752), ('tumpang', 567), ('danang', 241), ('periksa', 135), ('berangkat', 134)]

Top 5 untuk tag 'JJ':
[('september', 176), ('aman', 159), ('selamat', 120), ('sesuai', 119), ('internasional', 117)]

Top 5 untuk tag 'NNP':
[('of', 73), ('sabtu', 55), ('garuda', 32), ('haluoleo', 28), ('and', 8)]


NER with cahya bert (normalized_text)

In [23]:
# Function to extract named entities
def extract_hf_entities(text):
    try:
        results = ner_model(text)
        return [(r['word'], r['entity_group']) for r in results]
    except Exception as e:
        return []

df_lion["ner_tags"] = df_lion["text_final"].progress_apply(extract_hf_entities)


100%|██████████| 153/153 [03:02<00:00,  1.19s/it]


In [24]:
df_lion[["text_final", "ner_tags"]].head(30)

,text_final,ner_tags
0,kontancoid jakarta lion group dukung penuh bij...,"[(kontancoid jakarta lion group, ORG), (menter..."
1,peristiwa oktober salah satu jatuh pesawat lio...,"[(oktober, DAT), (pesawat lion air terbang, PR..."
2,sekretaris pp muhammadiyah abdul muti kritik p...,[]
3,larang keras ambil konten crawling indeks otom...,"[(indeks, PRD), (ai situs web izin tulis kanto..."
4,feb husni anggoro kali jakarta phu direktorat ...,[]
5,historical flight lion air bawa jalan lion ai...,"[(historical flight, EVT), (lion air, ORG), (l..."
6,adil negeri pn padang denda maskapai lion air ...,"[(adil, NOR), (negeri, LAW), (pn padang, ORG),..."
7,gelang emas cincin berat gram duga hilang curi...,[]
8,tumpang histeris hilang emas bandara karyawan ...,"[(lion air aman kendari, ORG), (kompascom, ORG..."
9,januari wib dengar artikel tempoco jakarta lio...,[]


In [25]:
# --- Analisis Frekuensi NER (Named Entity Recognition) ---

# Buat dictionary untuk menyimpan Counter untuk setiap tag NER
ner_counts = defaultdict(Counter)

# Iterasi setiap baris di DataFrame
# df_lion['ner_tags'] berisi list of tuples, misal: [('lion air', 'ORG')]
for list_of_entities in df_lion['ner_tags']:
    for entity, tag in list_of_entities:
        # Kita gunakan .lower() agar 'Lion Air' dan 'lion air' dihitung sama
        ner_counts[tag][entity.lower()] += 1

print("\n\n--- Frekuensi Top 5 Entitas per Jenis NER ---")

# Tampilkan 5 entitas paling umum untuk setiap tag NER
# (ORG=Organisasi, PER=Orang, LOC=Lokasi, GPE=Entitas Geo-Politik)
relevant_ner_tags = ['ORG', 'PER', 'LOC', 'GPE']

for tag in relevant_ner_tags:
    if tag in ner_counts:
        print(f"\nTop 5 untuk tag '{tag}':")
        print(ner_counts[tag].most_common(5))



--- Frekuensi Top 5 Entitas per Jenis NER ---

Top 5 untuk tag 'ORG':
[('lion air', 187), ('lion', 177), ('##ng', 43), ('lion air group', 34), ('jt', 34)]

Top 5 untuk tag 'PER':
[('ronald', 29), ('h', 28), ('dana', 18), ('daniel', 14), ('prihan', 12)]

Top 5 untuk tag 'LOC':
[('merauke', 10), ('garbarata', 6), ('jalan', 4), ('##uoleo', 4), ('##u', 4)]

Top 5 untuk tag 'GPE':
[('jakarta', 98), ('tanjungpinang', 36), ('makassar', 32), ('malaysia', 26), ('papua', 20)]


In [26]:
df_lion

,Kategori,Link,article_title,article_text,domain,cleaned_article_text,polarity,subjectivity,tokens,tokens_no_stop,tokens_final,tokens_stemmed,text_final,sentiment_label,pos_tags,ner_tags
0,Finance,https://industri.kontan.co.id/news/pemerintah-...,"Pemerintah Resmi Turunkan Harga Tiket Pesawat,...",Reporter: Leni Wandira | Editor: Wahyu T.Rahma...,kontan,kontancoid jakarta lion group mendukung penuh...,0.089297,0.302210,"['kontancoid', 'jakarta', 'lion', 'group', 'me...","['kontancoid', 'jakarta', 'lion', 'group', 'me...","['kontancoid', 'jakarta', 'lion', 'group', 'me...","['kontancoid', 'jakarta', 'lion', 'group', 'du...",kontancoid jakarta lion group dukung penuh bij...,Positive,"[(kontancoid, NN), (jakarta, NN), (lion, NN), ...","[(kontancoid jakarta lion group, ORG), (menter..."
1,Incident,https://news.okezone.com/read/2023/10/29/337/2...,Peristiwa 29 Oktober : Pesawat Lion Air Jatuh ...,SEJUMLAH peristiwa terjadi pada 29 Oktober. Sa...,okezone,sejumlah peristiwa terjadi pada oktober salah...,0.134921,0.250794,"['sejumlah', 'peristiwa', 'terjadi', 'pada', '...","['peristiwa', 'oktober', 'salah', 'satunya', '...","['peristiwa', 'oktober', 'salah', 'satunya', '...","['peristiwa', 'oktober', 'salah', 'satu', 'jat...",peristiwa oktober salah satu jatuh pesawat lio...,Positive,"[(peristiwa, NN), (oktober, FW), (salah, JJ), ...","[(oktober, DAT), (pesawat lion air terbang, PR..."
2,Incident,https://www.cnnindonesia.com/ekonomi/202306081...,Deret Masalah Penerbangan Lion Air Group Sepan...,Sekretaris Umum PP Muhammadiyah Abdul Mu'ti me...,cnnindonesia,sekretaris umum pp muhammadiyah abdul muti men...,0.033559,0.340945,"['sekretaris', 'umum', 'pp', 'muhammadiyah', '...","['sekretaris', 'pp', 'muhammadiyah', 'abdul', ...","['sekretaris', 'pp', 'muhammadiyah', 'abdul', ...","['sekretaris', 'pp', 'muhammadiyah', 'abdul', ...",sekretaris pp muhammadiyah abdul muti kritik p...,Netral,"[(sekretaris, NN), (pp, NN), (muhammadiyah, JJ...",[]
3,Incident,https://www.antaranews.com/video/3366141/ini-d...,Ini dugaan penyebab kecelakaan pesawat Lion Ai...,Copyright © ANTARA 2023\nDilarang keras mengam...,antaranews,dilarang keras mengambil konten melakukan craw...,0.000000,0.000000,"['dilarang', 'keras', 'mengambil', 'konten', '...","['dilarang', 'keras', 'mengambil', 'konten', '...","['dilarang', 'keras', 'mengambil', 'konten', '...","['larang', 'keras', 'ambil', 'konten', 'crawli...",larang keras ambil konten crawling indeks otom...,Netral,"[(larang, NN), (keras, JJ), (ambil, VB), (kont...","[(indeks, PRD), (ai situs web izin tulis kanto..."
4,Operational,https://haji.kemenag.go.id/v5/detail/tingkatka...,"Tingkatkan Kualitas Layanan, Kemenag - Lion Ai...",21 Feb 2025 oleh Husni Anggoro | dilihat 42259...,kemenag,feb oleh husni anggoro dilihat kali\njakar...,0.144516,0.421348,"['feb', 'oleh', 'husni', 'anggoro', 'dilihat',...","['feb', 'husni', 'anggoro', 'kali', 'jakarta',...","['feb', 'husni', 'anggoro', 'kali', 'jakarta',...","['feb', 'husni', 'anggoro', 'kali', 'jakarta',...",feb husni anggoro kali jakarta phu direktorat ...,Positive,"[(feb, NN), (husni, VB), (anggoro, NN), (kali,...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,NaN,https://news.okezone.com/read/2018/11/03/337/1...,Penjelasan Mantan Pilot Terkait Hancurnya Pesa...,\nJAKARTA - Mantan pilot yang juga Purnawirawa...,okezone,jakarta mantan pilot yang juga purnawirawan k...,-0.056875,0.360000,"['jakarta', 'mantan', 'pilot', 'yang', 'juga',...","['jakarta', 'mantan', 'pilot', 'purnawirawan',...","['jakarta', 'mantan', 'pilot', 'purnawirawan',...","['jakarta', 'mantan', 'pilot', 'purnawirawan',...",jakarta mantan pilot purnawirawan polisi kapte...,Negative,"[(jakarta, NN), (mantan, JJ), (pilot, NN), (pu...","[(jakarta, GPE), (polisi, NOR), (kapten john b..."
149,NaN,https://www.antaranews.com/berita/481275/lion-...,Lion Air klaim finansial maskapai masih stabil...,\nDua hari lalu terjadi kekacauan masif di pul...,antaranews,dua hari lalu terjadi keka

In [27]:
# Save the DataFrame to a CSV file
df_lion.to_csv('tagged_articles.csv', index=False)